In [2]:
# Necessary Libraries
import numpy as np
import os
import pathlib
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from tensorflow import keras
import tensorflow as tf

In [4]:
# Define Constants
IMAGE_SHAPE = (200, 200)
DATA_DIR = 'C:/Users/Indhu/Downloads/dog_breed_30_image/'  # Update with your dataset path
BATCH_SIZE = 100


In [6]:
# Load Data
data_dir = pathlib.Path(DATA_DIR)
breeds = os.listdir(DATA_DIR)
dogs_images_dict = {}
dogs_labels_dict = {}

In [9]:
# Organize Data by Category
for index, category in enumerate(breeds):
    dogs_images_dict[category] = list(data_dir.glob(category + '/*'))
    dogs_labels_dict[category] = index

In [11]:
X, y = [], []
for breed_name, images in dogs_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(dogs_labels_dict[breed_name])

In [13]:
X = np.array(X)
y = np.array(y)

In [15]:
# Normalize the Images
X = X / 255.0

In [17]:
# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Build Feature Extraction Model (Pretrained CNN)
base_model = keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the convolutional base

C:\Users\Indhu\AppData\Local\Temp\ipykernel_5812\2597365918.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 6s 1us/step


In [21]:
# Create the Feature Extraction Pipeline
feature_extractor = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D()
])


In [23]:
# Extract Features
X_train_features = feature_extractor.predict(X_train, batch_size=BATCH_SIZE, verbose=1)
X_test_features = feature_extractor.predict(X_test, batch_size=BATCH_SIZE, verbose=1)

38/38 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step


In [33]:
# Define Classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine": SVC(kernel='rbf', probability=True)
}

In [39]:
# Train and Evaluate Classifiers
for name, clf in classifiers.items():
    print(f"Training {name}...")
    clf.fit(X_train_features, y_train)
    y_pred = clf.predict(X_test_features)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    
    # Generate classification report
    print(classification_report(y_test, y_pred, target_names=breeds))

Training Random Forest...
Random Forest Accuracy: 0.7357
                             precision    recall  f1-score   support

                      boxer       0.65      0.46      0.54        37
          Brabancon_griffon       0.89      0.82      0.85        38
                     briard       0.77      0.74      0.75        31
           Brittany_spaniel       0.69      0.56      0.62        36
               bull_mastiff       0.53      0.63      0.58        27
                      cairn       0.83      0.81      0.82        42
                   Cardigan       0.72      0.64      0.68        36
   Chesapeake_Bay_retriever       0.59      0.61      0.60        36
                  Chihuahua       0.68      0.58      0.62        26
                       chow       0.84      0.95      0.89        44
                    clumber       0.61      0.70      0.66        27
             cocker_spaniel       0.79      0.50      0.61        38
                     collie       0.72      0

C:\Users\Indhu\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Indhu\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Indhu\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Indhu\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Indhu\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

K-Nearest Neighbors Accuracy: 0.7594
                             precision    recall  f1-score   support

                      boxer       0.67      0.59      0.63        37
          Brabancon_griffon       0.91      0.84      0.88        38
                     briard       0.77      0.74      0.75        31
           Brittany_spaniel       0.62      0.56      0.59        36
               bull_mastiff       0.54      0.81      0.65        27
                      cairn       0.83      0.90      0.86        42
                   Cardigan       0.76      0.72      0.74        36
   Chesapeake_Bay_retriever       0.53      0.67      0.59        36
                  Chihuahua       0.83      0.58      0.68        26
                       chow       0.98      0.98      0.98        44
                    clumber       0.63      0.96      0.76        27
             cocker_spaniel       0.89      0.63      0.74        38
                     collie       0.79      0.85      0.81       